<a href="https://colab.research.google.com/github/Nb4159/TreeTensorMNISTClassifier/blob/main/MNIST_Classification_using_Quantum_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies

In [1]:
! pip install tensorflow-quantum

In [2]:
!pip install cirq

  Using cached cirq_core-1.4.1-py3-none-any.whl (1.9 MB)
  Using cached cirq_google-1.4.1-py3-none-any.whl (532 kB)
  Attempting uninstall: cirq-core
    Found existing installation: cirq-core 1.3.0
    Uninstalling cirq-core-1.3.0:
      Successfully uninstalled cirq-core-1.3.0
  Attempting uninstall: cirq-google
    Found existing installation: cirq-google 1.3.0
    Uninstalling cirq-google-1.3.0:
      Successfully uninstalled cirq-google-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-quantum 0.7.3 requires cirq-core==1.3.0, but you have cirq-core 1.4.1 which is incompatible.
tensorflow-quantum 0.7.3 requires cirq-google==1.3.0, but you have cirq-google 1.4.1 which is incompatible.


# Data

In [3]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Here I keep only digits 0-3
train_mask = (y_train < 4)
test_mask = (y_test < 4)
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# Dimensionality reduction using PCA
pca = PCA(n_components=8)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

# Scale features
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train_pca)
x_test_scaled = scaler.transform(x_test_pca)

x_train_scaled



array([[0.6575422 , 0.5113609 , 0.15022027, ..., 0.52433723, 0.3400782 ,
        0.468922  ],
       [0.08702649, 0.66288257, 0.42998007, ..., 0.50262123, 0.5097608 ,
        0.42599776],
       [0.3944188 , 0.74505955, 0.47534764, ..., 0.39494434, 0.706673  ,
        0.47034067],
       ...,
       [0.30617657, 0.8178096 , 0.28421837, ..., 0.19113496, 0.43874547,
        0.39723587],
       [0.09423779, 0.5708636 , 0.31920433, ..., 0.30451584, 0.46896046,
        0.37133288],
       [0.3725143 , 0.49605983, 0.61925894, ..., 0.18966365, 0.3179507 ,
        0.69953877]], dtype=float32)

# Amplitude Encoding
This is like making an embedding to feed data to the classifier in appropriate way. Amplitude Encoding encodes 2^n features into amplitude vector of n qubits.

# Tree Tensor Network[(TTN)](https://https://arxiv.org/abs/1803.11537)
TTNs are a type of tensor network that organizes tensors in a tree-like structure. They're used to represent quantum states efficiently.
Some of its features are:

**Hierarchical structure**: Information flows from the bottom (input) to the top (output).

**Logarithmic depth**: The number of layers scales logarithmically with the number of input qubits.

**Entanglement structure**: TTNs can capture short-range entanglement well but may struggle with long-range entanglement.



In [4]:
# Amplitude encoding
def amplitude_encode(features):
    qubits = cirq.GridQubit.rect(1, 8)
    circuit = cirq.Circuit()
    for i, feature in enumerate(features):
        circuit.append(cirq.ry(np.arccos(feature) * 2)(qubits[i]))
    return circuit

x_train_circuits = [amplitude_encode(x) for x in x_train_scaled]
x_test_circuits = [amplitude_encode(x) for x in x_test_scaled]

# Define the TTN model
def create_ttn_model():
    qubits = cirq.GridQubit.rect(1, 8)
    circuit = cirq.Circuit()

    # First layer
    for i in range(0, 8, 2):
        circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i}'))(qubits[i]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i+1}'))(qubits[i+1]))

    # Second layer
    for i in range(0, 8, 4):
        circuit.append(cirq.CNOT(qubits[i], qubits[i+2]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i+8}'))(qubits[i]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i+9}'))(qubits[i+2]))

    # Third layer
    circuit.append(cirq.CNOT(qubits[0], qubits[4]))
    circuit.append(cirq.ry(sympy.Symbol('θ_16'))(qubits[0]))
    circuit.append(cirq.ry(sympy.Symbol('θ_17'))(qubits[4]))

    # Measurement
    circuit.append(cirq.measure(qubits[0], qubits[4], key='m'))

    readout_ops = [cirq.Z(qubits[0]), cirq.Z(qubits[4])]  # Readout operators

    pqc = tfq.layers.PQC(circuit, readout_ops)
    input_tensor = tf.keras.Input(shape=(), dtype=tf.string)
    output = pqc(input_tensor)
    output = tf.keras.layers.Dense(4, activation='softmax')(output)  # 4 output classes

    model = tf.keras.Model(inputs=[input_tensor], outputs=[output])

    return model, circuit

ttn_model,qc = create_ttn_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
ttn_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])




<ipython-input-4-d66c714e080e>:6: RuntimeWarning: invalid value encountered in arccos
  circuit.append(cirq.ry(np.arccos(feature) * 2)(qubits[i]))


In [5]:
from tensorflow.keras.utils import plot_model
import os
plot_model(ttn_model, to_file='ttn_model.png', show_shapes=True, show_layer_names=True)

circuit_diagram = cirq.Circuit(qc)
print(circuit_diagram)


(0, 0): ───@───Ry(θ_0)───@───Ry(θ_8)────@───Ry(θ_16)───M('m')───
           │             │              │              │
(0, 1): ───X───Ry(θ_1)───┼──────────────┼──────────────┼────────
                         │              │              │
(0, 2): ───@───Ry(θ_2)───X───Ry(θ_9)────┼──────────────┼────────
           │                            │              │
(0, 3): ───X───Ry(θ_3)──────────────────┼──────────────┼────────
                                        │              │
(0, 4): ───@───Ry(θ_4)───@───Ry(θ_12)───X───Ry(θ_17)───M────────
           │             │
(0, 5): ───X───Ry(θ_5)───┼──────────────────────────────────────
                         │
(0, 6): ───@───Ry(θ_6)───X───Ry(θ_13)───────────────────────────
           │
(0, 7): ───X───Ry(θ_7)──────────────────────────────────────────


As we can see our neural network is in the form of a tree.

In [6]:
x_train_tfq = tfq.convert_to_tensor(x_train_circuits)
x_test_tfq = tfq.convert_to_tensor(x_test_circuits)

history = ttn_model.fit(
    x_train_tfq, y_train,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    verbose=1
)


test_loss, test_accuracy = ttn_model.evaluate(x_test_tfq, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


619/619 [==============================] - 68s 104ms/step - loss: 0.9341 - accuracy: 0.6299 - val_loss: 0.7190 - val_accuracy: 0.7613
Epoch 2/50
619/619 [==============================] - 63s 101ms/step - loss: 0.6615 - accuracy: 0.7569 - val_loss: 0.6064 - val_accuracy: 0.7786
Epoch 3/50
619/619 [==============================] - 61s 99ms/step - loss: 0.6016 - accuracy: 0.7681 - val_loss: 0.5711 - val_accuracy: 0.7978
Epoch 4/50
619/619 [==============================] - 64s 103ms/step - loss: 0.5735 - accuracy: 0.7718 - val_loss: 0.5466 - val_accuracy: 0.7861
Epoch 5/50
619/619 [==============================] - 57s 93ms/step - loss: 0.5572 - accuracy: 0.7760 - val_loss: 0.5305 - val_accuracy: 0.7930
Epoch 6/50
619/619 [==============================] - 61s 99ms/step - loss: 0.5491 - accuracy: 0.7764 - val_loss: 0.5327 - val_accuracy: 0.7837
Epoch 7/50
619/619 [==============================] - 61s 99ms/step - loss: 0.5395 - accuracy: 0.7798 - val_loss: 0.5188 - val_accuracy: 0.7950
